# Statoil/C-CORE Iceberg Classifier Challenge

Link: https://www.kaggle.com/c/statoil-iceberg-classifier-challenge


This Kernal implements a Keras + Tensorflow CNN for the StatOil Iceberg competition. It has yielded results of 0.1995 on the leaderboard. With some tuning and image filtering plus more of an inclusion of the incident angle, a better result could be yielded I'm sure.

The input is a 75x75x3 set of images. The output is a binary 0/1 where 1 is noteed as an iceberg. 

The set of images are band_1 (HH), band_2 (HV), and an combined band which would be (HH dot HV)/constant. However, since we are working with the images in dB, the 3rd band is modified to compenate for the log function yielding band_1 + band_2 -log(constant). The last term is neglected as when the images are scaled the 3rd term would be removed by the mathematics anyway.

This and other information can be found from: https://earth.esa.int/c/document_library/get_file?folderId=409229&name=DLFE-5566.pdf

## 1. Imports packages

How to use GPU on google drive: https://zhuanlan.zhihu.com/p/33344222

In [0]:
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
!pip install keras


    100% |████████████████████████████████| 327kB 2.1MB/s 


In [0]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # Set the seed to generate same random sequence

# Import Keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.optimizers import Adam

In [0]:
def limit_men():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
limit_men()

## 2.Data engineering

### 2.1 Load Training Data


In [17]:
import io
from google.colab import files
uploaded = files.upload()
df_train = pd.read_json(io.StringIO(uploaded['train.json'].decode('utf-8')))

KeyError: ignored

![alt text](https://)The database contains 1604 data sample and five features: band 1, band 2 , id , inc angle, is iceberg


In [0]:
print ( "The shape of database is :", df_train.shape) 
print( "Show the first three examples")
df_train[:3]
print("How many pixels in the each chart",len(df_train["band_1"][0]), 
      "which is equalt to 75*75")


The shape of database is : (1604, 5)
Show the first three examples
How many pixels in the each chart 5625 which is equalt to 75*75


Need to reshape and feature scale the images: reshape the chart into [75 * 75] and scale each pixel with max- min

In [0]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

Get the train data

In [0]:
Xtrain_orig = get_scaled_imgs(df_train)
print( "The shape after reshape is:", Xtrain_orig.shape, 
      " The three dimension is band 1, band 2 and (band 1+ band 2)/2")

The shape after reshape is: (1604, 75, 75, 3)  The three dimension is band 1, band 2 and (band 1+ band 2)/2


Get the response variable "is_iceberg": to verify if the chart contains ice_berg

In [0]:
Ytrain_orig = np.array(df_train['is_iceberg'])

Deal with the missing value: Some of the incident angle from the satellite are unknown and marked as "na". Replace these na with 0 and find the indices where the incident angle is >0 (this way you can use a truncated set or the full set of training data). negelect all the "na" data in the data sample

In [0]:
df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

You can now use the option of training with only known incident angles or the whole set. I found slightly better results training with only the known incident angles so:

In [0]:
Ytrain = Ytrain_orig[idx_tr[0]]
Xtrain = Xtrain_orig[idx_tr[0],...]
print("The remaining shape after dealing with missing value is :", Xtrain.shape)

The remaining shape after dealing with missing value is : (1471, 75, 75, 3)


### 2.2 Adding images for training

Now, the biggest improvement I had was by adding more data to train on. I did this by simply including horizontally and vertically flipped data. Using OpenCV this is easily done. The final data will contain original chart, vertical chart  and horizontal chart

link:https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#void 

Parameters:	
src – input array.

dst – output array of the same size and type as src.
flipCode – a flag to specify how to flip the array; 0 means flipping around the x-axis and positive value (for example, 1) means flipping around y-axis. Negative value (for example, -1) means flipping around both axes (see the discussion below for the formulas).

In [0]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        # vertical rotate
        av=cv2.flip(a,1)
        # horizontal rotate
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images


I rename the returned value so i have the option of using the original data set or the expanded data set

In [0]:
Xtr_more = get_more_images(Xtrain) 

And then define the new response variable: response is just the three times of the original data, since the chart rotation will not influnce the results of responses

In [0]:
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

## 3. CNN Keras Model

Convolutional neural networks (CNNs) are the current state-of-the-art model architecture for image classification tasks. CNNs apply a series of filters to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains three components:

**Convolutional layers**, which apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. Convolutional layers then typically apply a ReLU activation function to the output to introduce nonlinearities into the model.

**Pooling layers**, which downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values.

**Dense (fully connected) layers**, which perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.

Example of a CNN:

<img src="../images/cnn.png" />

Now the nitty gritty of the situation, the CNN model. The model contains 4 cnn layers and 2 dense layers.

The link about sequential model:https://keras.io/getting-started/sequential-model-guide/

Link of convolutional layers: https://keras.io/layers/convolutional/

Standford convulutinal neural network link:http://cs231n.github.io/convolutional-networks/

Default filters glorot uniform: https://keras.io/initializers/

Optimization function: http://ruder.io/optimizing-gradient-descent/index.html#adam

Activation function :https://en.wikipedia.org/wiki/Activation_function

Some important parameters:

filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).

kernel_size: An integer or tuple/list of a single integer, specifying the length of the 1D convolution window.

strides: An integer or tuple/list of a single integer, specifying the stride length of the convolution. Specifying any stride value != 1 is incompatible with specifying any  dilation_rate value != 1.

padding: One of "valid", "causal" or "same" (case-insensitive). "valid" means "no padding". "same" results in padding the input such that the output has the same length as the original input. "causal" results in causal (dilated) convolutions, e.g. output[t] does not depend on input[t+1:]. Useful when modeling temporal data where the model should not violate the temporal order. See WaveNet: A Generative Model for Raw Audio, section 2.1.

dilation_rate: an integer or tuple/list of a single integer, specifying the dilation rate to use for dilated convolution. Currently, specifying any dilation_rate value != 1 is incompatible with specifying any strides value != 1.

activation: Activation function to use (see activations). If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).





In [0]:
def getModel():
    #Build keras model
    
    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(filtes=64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers: nakes it one dimension
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

Now get the model and get ready to train

In [0]:
model = getModel()
model.summary()

batch_size = 32
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, 
                                   epsilon=1e-4, mode='min')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

Now train the model! (Each epoch ran at about 10s on GPU)

In [0]:
model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=1, 
          callbacks=[mcp_save, reduce_lr_loss], validation_split=0.25)

Train on 3309 samples, validate on 1104 samples
Epoch 1/50
3309/3309 [==============================] - 46s 14ms/step - loss: 0.5589 - acc: 0.7014 - val_loss: 0.4020 - val_acc: 0.8288
Epoch 2/50
3309/3309 [==============================] - 45s 14ms/step - loss: 0.4087 - acc: 0.8154 - val_loss: 0.5990 - val_acc: 0.7120
Epoch 3/50
3309/3309 [==============================] - 45s 14ms/step - loss: 0.3281 - acc: 0.8516 - val_loss: 0.3425 - val_acc: 0.8433
Epoch 4/50
3309/3309 [==============================] - 45s 14ms/step - loss: 0.2746 - acc: 0.8803 - val_loss: 0.2508 - val_acc: 0.9004
Epoch 5/50
3309/3309 [==============================] - 45s 14ms/step - loss: 0.2522 - acc: 0.8921 - val_loss: 0.3039 - val_acc: 0.8668
Epoch 6/50
3309/3309 [==============================] - 46s 14ms/step - loss: 0.2273 - acc: 0.9036 - val_loss: 0.2306 - val_acc: 0.9139
Epoch 7/50
3309/3309 [==============================] - 45s 14ms/step - loss: 0.2077 - acc: 0.9102 - val_loss: 0.2291 - val_acc: 0.9212


## Results

Load the best weights and check the score on the training data.

In [0]:
model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1471/1471 [==============================] - 5s 4ms/step
Train score: 0.104133588701
Train accuracy: 0.963970087767


## Psuedo labelling 

Use the model to predict the test data and use the result as label to retrain the train+test data. Use the retrained model with psuedo labeling data to predict the test sample. First predict the classes for the test data.

In [0]:
df_test = pd.read_json('../data/test.json')
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
pred_test_classes = model.predict_classes(Xtest)

NameError: name 'get_scaled_imgs' is not defined

Next retrain the model with psuedo labeling test data+ train data


In [0]:
x_train_total=np.concatenate((Xtest,Xtr_more),axis=0)
y_train_total=np.concatenate((Ytr_more[:,np.newaxis],pred_test_classes),axis =0)

IndexError: axis 1 out of bounds [0, 1)

In [0]:
import time 
tic=time.time()
model.fit(x_train_total, y_train_total, batch_size=batch_size, epochs=50, 
          verbose=1, callbacks=[mcp_save, reduce_lr_loss], validation_split=0.25)
print(time.time()-tic)

Train on 9627 samples, validate on 3210 samples
Epoch 1/50
9627/9627 [==============================] - 134s 14ms/step - loss: 2.4336 - acc: 0.5006 - val_loss: 3.2997 - val_acc: 0.4754
Epoch 2/50
9627/9627 [==============================] - 139s 14ms/step - loss: 2.4359 - acc: 0.4943 - val_loss: 3.2951 - val_acc: 0.4757
Epoch 3/50
9627/9627 [==============================] - 135s 14ms/step - loss: 2.4090 - acc: 0.4962 - val_loss: 3.2913 - val_acc: 0.4757
Epoch 4/50
9627/9627 [==============================] - 139s 14ms/step - loss: 2.4168 - acc: 0.4953 - val_loss: 3.2878 - val_acc: 0.4757
Epoch 5/50
9627/9627 [==============================] - 136s 14ms/step - loss: 2.4007 - acc: 0.4965 - val_loss: 3.2846 - val_acc: 0.4757
Epoch 6/50
9627/9627 [==============================] - 139s 14ms/step - loss: 2.4189 - acc: 0.4993 - val_loss: 3.2815 - val_acc: 0.4760
Epoch 7/50
9627/9627 [==============================] - 138s 14ms/step - loss: 2.4125 - acc: 0.4982 - val_loss: 3.2785 - val_acc: 

Use the new model to predict the test data, fisrt to see the train accuaracy and then the test result


In [0]:
score = model.evaluate(x_train_total, y_train_total, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

pred_test = model.predict(Xtest)

12837/12837 [==============================] - 49s 4ms/step
Train score: 2.44602600345
Train accuracy: 0.491859468726


Now, to make a submission, load the test data and train the model and output a csv file.

In [0]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

submission.to_csv('submission.csv', index=False)

         id  is_iceberg
0  5941774d    0.025917
1  4023181e    0.955667
2  b20200e4    0.028548
3  e7f018bb    0.999707
4  4371c8c3    0.923421
5  a8d9b1fd    0.748707
6  29e7727e    0.017642
7  92a51ffb    0.999460
8  c769ac97    0.000126
9  aee0547d    0.000007


The best submission with this I received was 0.1813 on the leaderboard. Have a go and see how well you can do!

## Test data augument
Try to use more test data,  by means of flip, and test the diferent test data in diferent time, finally take a stack resulting of different test data set.

In [0]:
from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
# for band 1


In [0]:
def get_scaled_imgs(dict):
    imgs = []
    
    for i in range (len(dict["band_1"])):
        #make 75x75 image
        band_1 = np.array(dict["band_1"][i]).reshape(75, 75)
        band_2 = np.array(dict["band_2"][i]).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [0]:
# for 45 rotation
gin_45={}
gin_45["band_1"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_1']])
gin_45["band_1"] = gin_45["band_1"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=45)
datagen.fit(gin_45["band_1"])
gin_45["band_2"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_2']])
gin_45["band_2"] = gin_45["band_2"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=45)
datagen.fit(gin_45["band_2"])

In [0]:
test_x_45=get_scaled_imgs(gin_45)

In [0]:
# for 90 rotation
gin_90={}
gin_90["band_1"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_1']])
gin_90["band_1"] = gin_90["band_1"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(gin_90["band_1"])
gin_90["band_2"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_2']])
gin_90["band_2"] = gin_90["band_2"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(gin_90["band_2"])

In [0]:
test_x_90=get_scaled_imgs(gin_90)

In [0]:
# for 135 rotation
gin_135={}
gin_135["band_1"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_1']])
gin_135["band_1"] = gin_135["band_1"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=135)
datagen.fit(gin_135["band_1"])
gin_135["band_2"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_2']])
gin_135["band_2"] = gin_135["band_2"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=135)
datagen.fit(gin_135["band_2"])

In [0]:
test_x_135=get_scaled_imgs(gin_135)

In [0]:
# for 135 rotation
gin_180={}
gin_180["band_1"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_1']])
gin_180["band_1"] = gin_180["band_1"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=135)
datagen.fit(gin_180["band_1"])
gin_180["band_2"] = np.asarray([np.asarray(p).reshape(75,75) for p in df_test['band_2']])
gin_180["band_2"] = gin_180["band_2"].reshape(8424,75,75,1)
datagen = ImageDataGenerator(rotation_range=135)
datagen.fit(gin_180["band_2"])

In [0]:
test_x_180=get_scaled_imgs(gin_180)

In [0]:
test_x_180-test_x_135

array([[[[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        ..., 
        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         ..., 
         [ 0.,  0.,  0.],
  